In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

In [2]:
current_dir = os.getcwd() # Obtener la ruta del directorio actual del notebook
ROOT_PATH = os.path.dirname(current_dir) # Obtener la ruta del directorio superior
sys.path.insert(1, ROOT_PATH) # Insertar la ruta en sys.path

import root 

In [3]:
#creating paths from root 

client_path = root.DIR_DATA_RAW + 'client.csv'
electricity_prices_path = root.DIR_DATA_RAW + 'electricity_prices.csv'
gas_prices_path = root.DIR_DATA_RAW + 'gas_prices.csv'
historical_weather_path = root.DIR_DATA_RAW + 'hist_weather_processed.csv'
train_path_c = root.DIR_DATA_RAW + 'train_consumption.csv'
train_path_p = root.DIR_DATA_RAW + 'train_production.csv'

In [4]:
# Read datasets from local
client = pd.read_csv(client_path)
electricity_prices = pd.read_csv(electricity_prices_path)
gas_prices = pd.read_csv(gas_prices_path)
historical_weather = pd.read_csv(historical_weather_path)
train_c = pd.read_csv(train_path_c)
train_p = pd.read_csv(train_path_p)

In [5]:
train_p.isnull().sum()

county       0
target      64
datetime     0
dtype: int64

In [6]:
train_p = train_p[train_p['county'] == 0]
train_p['datetime'] = pd.to_datetime(train_p['datetime'])
train_p['date'] = train_p['datetime'].dt.floor('D')
train_p.head()

,county,target,datetime,date
0,0,0.315,2021-09-01 00:00:00,2021-09-01
16,0,0.353,2021-09-01 01:00:00,2021-09-01
32,0,0.338,2021-09-01 02:00:00,2021-09-01
48,0,0.303,2021-09-01 03:00:00,2021-09-01
64,0,0.424,2021-09-01 04:00:00,2021-09-01


In [7]:
train_p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15312 entries, 0 to 244976
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   county    15312 non-null  int64         
 1   target    15308 non-null  float64       
 2   datetime  15312 non-null  datetime64[ns]
 3   date      15312 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 598.1 KB


In [8]:
gas_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   forecast_date          637 non-null    object 
 1   lowest_price_per_mwh   637 non-null    float64
 2   highest_price_per_mwh  637 non-null    float64
 3   origin_date            637 non-null    object 
 4   data_block_id          637 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 25.0+ KB


In [9]:
train_c = train_c[train_c['county'] == 0]
train_c['datetime'] = pd.to_datetime(train_c['datetime'])
train_c['date'] = train_c['datetime'].dt.floor('D')
train_c.head()

,county,target,datetime,date
0,0,5155.056,2021-09-01 00:00:00,2021-09-01
16,0,4953.258,2021-09-01 01:00:00,2021-09-01
32,0,4910.692,2021-09-01 02:00:00,2021-09-01
48,0,4700.725,2021-09-01 03:00:00,2021-09-01
64,0,4857.681,2021-09-01 04:00:00,2021-09-01


In [10]:
client = client[client['county'] == 0]
client = client[client['product_type'] == 3]
client['date'] = pd.to_datetime(client['date'])
client['date_2_join'] = pd.to_datetime(client['date']) + pd.Timedelta(days=3)
client.drop(columns=['county','is_business','date','data_block_id','product_type'], inplace=True)
client.head()

,eic_count,installed_capacity,date_2_join
2,688,7207.88,2021-09-04
5,265,13417.79,2021-09-04
63,688,7207.88,2021-09-05
66,265,13417.79,2021-09-05
124,688,7207.88,2021-09-06


In [11]:
electricity_prices.drop(columns=['origin_date','data_block_id'], inplace=True)
electricity_prices['forecast_date'] = pd.to_datetime(electricity_prices['forecast_date'])
electricity_prices['date_2_join'] = pd.to_datetime(electricity_prices['forecast_date']) + pd.Timedelta(days=1)
electricity_prices.drop(columns=['forecast_date'], inplace=True)
electricity_prices.head()

,euros_per_mwh,date_2_join
0,92.51,2021-09-02 00:00:00
1,88.90,2021-09-02 01:00:00
2,87.35,2021-09-02 02:00:00
3,86.88,2021-09-02 03:00:00
4,88.43,2021-09-02 04:00:00


In [12]:
gas_prices.drop(columns=['origin_date','data_block_id'], inplace=True)
gas_prices['forecast_date'] = pd.to_datetime(gas_prices['forecast_date'])
gas_prices['date_2_join'] = pd.to_datetime(gas_prices['forecast_date']) + pd.Timedelta(days=1)
gas_prices.drop(columns=['forecast_date'], inplace=True)
gas_prices.head()

,lowest_price_per_mwh,highest_price_per_mwh,date_2_join
0,45.23,46.32,2021-09-02
1,45.62,46.29,2021-09-03
2,45.85,46.40,2021-09-04
3,46.30,46.80,2021-09-05
4,46.30,46.58,2021-09-06


In [13]:
historical_weather = historical_weather[historical_weather['county'] == 0]
historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
historical_weather['date_2_join'] = historical_weather['datetime'] + pd.Timedelta(days=2)
historical_weather.drop(columns=['datetime','date'], inplace=True)
historical_weather.head()

,county,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,date_2_join
0,0,13.79,10.73,0.0,0.0,1010.56,54.4,56.9,3.5,3.9,5.325000,342.0,0.0,0.0,0.0,2021-09-03 00:00:00
1,0,13.62,10.35,0.0,0.0,1010.47,28.9,26.0,1.1,15.8,5.327778,345.8,0.0,0.0,0.0,2021-09-03 01:00:00
2,0,13.13,10.07,0.0,0.0,1010.14,13.7,14.5,0.1,1.8,5.091667,340.8,0.0,0.0,0.0,2021-09-03 02:00:00
3,0,12.74,9.94,0.0,0.0,1010.09,8.6,6.5,0.9,7.5,4.944444,335.5,0.0,0.0,0.0,2021-09-03 03:00:00
4,0,12.46,10.03,0.0,0.0,1009.83,4.1,2.8,0.6,4.2,4.669444,330.6,0.0,0.0,0.0,2021-09-03 04:00:00


## Mergin data sets

## Merged consumption

In [18]:
merged_df = train_c.merge(historical_weather, left_on=['datetime', 'county'], right_on=['date_2_join', 'county'], how='left')
merged_df.drop(columns=['date_2_join'], inplace=True)
merged_df = merged_df.merge(gas_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df.drop(columns=['date_2_join'], inplace=True)
merged_df = merged_df.merge(electricity_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df.drop(columns=['date_2_join','county'], inplace=True)
merged_df = merged_df.merge(client, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df.drop(columns=['date_2_join'], inplace=True)

In [20]:
# Define el nuevo orden de las columnas
column_order = ['datetime', 'date', 'target', 'temperature', 'dewpoint', 'rain', 'snowfall',
                'surface_pressure', 'cloudcover_total', 'cloudcover_low', 'cloudcover_mid', 
                'cloudcover_high', 'windspeed_10m', 'winddirection_10m', 
                'shortwave_radiation', 'direct_solar_radiation', 'diffuse_radiation', 
                'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh','eic_count', 'installed_capacity']

# Reordena el DataFrame
merged_df = merged_df[column_order]


In [21]:
merged_df.columns

Index(['datetime', 'date', 'target', 'temperature', 'dewpoint', 'rain',
       'snowfall', 'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'euros_per_mwh', 'eic_count', 'installed_capacity'],
      dtype='object')

In [22]:
merged_df.head()

,datetime,date,target,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,...,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity
0,2021-09-01 00:00:00,2021-09-01,5155.056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-01 01:00:00,2021-09-01,4953.258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-01 02:00:00,2021-09-01,4910.692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-01 03:00:00,2021-09-01,4700.725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-01 04:00:00,2021-09-01,4857.681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
merged_df.isnull().sum()

datetime                   0
date                       0
target                     8
temperature               48
dewpoint                  48
rain                      48
snowfall                  48
surface_pressure          48
cloudcover_total          48
cloudcover_low            48
cloudcover_mid            48
cloudcover_high           48
windspeed_10m             48
winddirection_10m         48
shortwave_radiation       48
direct_solar_radiation    48
diffuse_radiation         48
lowest_price_per_mwh      24
highest_price_per_mwh     24
euros_per_mwh             24
eic_count                 72
installed_capacity        72
dtype: int64

## Merged Production

In [24]:
merged_df_p = train_p.merge(historical_weather, left_on=['datetime', 'county'], right_on=['date_2_join', 'county'], how='left')
merged_df_p.drop(columns=['date_2_join'], inplace=True)
merged_df_p = merged_df_p.merge(gas_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df_p.drop(columns=['date_2_join'], inplace=True)
merged_df_p = merged_df_p.merge(electricity_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df_p.drop(columns=['date_2_join','county'], inplace=True)
merged_df_p = merged_df_p.merge(client, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df_p.drop(columns=['date_2_join'], inplace=True)

In [25]:
# Define el nuevo orden de las columnas
column_order = ['datetime', 'date', 'target', 'temperature', 'dewpoint', 'rain', 'snowfall',
                'surface_pressure', 'cloudcover_total', 'cloudcover_low', 'cloudcover_mid', 
                'cloudcover_high', 'windspeed_10m', 'winddirection_10m', 
                'shortwave_radiation', 'direct_solar_radiation', 'diffuse_radiation', 
                'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh','eic_count', 'installed_capacity']

# Reordena el DataFrame
merged_df_p = merged_df_p[column_order]


In [26]:
merged_df_p.columns

Index(['datetime', 'date', 'target', 'temperature', 'dewpoint', 'rain',
       'snowfall', 'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'euros_per_mwh', 'eic_count', 'installed_capacity'],
      dtype='object')

In [27]:
merged_df_p.head()

,datetime,date,target,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,...,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity
0,2021-09-01 00:00:00,2021-09-01,0.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-01 01:00:00,2021-09-01,0.353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-01 02:00:00,2021-09-01,0.338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-01 03:00:00,2021-09-01,0.303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-01 04:00:00,2021-09-01,0.424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
merged_df_p.isnull().sum()

datetime                   0
date                       0
target                     8
temperature               48
dewpoint                  48
rain                      48
snowfall                  48
surface_pressure          48
cloudcover_total          48
cloudcover_low            48
cloudcover_mid            48
cloudcover_high           48
windspeed_10m             48
winddirection_10m         48
shortwave_radiation       48
direct_solar_radiation    48
diffuse_radiation         48
lowest_price_per_mwh      24
highest_price_per_mwh     24
euros_per_mwh             24
eic_count                 72
installed_capacity        72
dtype: int64